In [1]:
# The Google Earth Engine module
import ee

# The datetime module is used to specify the dates
# to search for imagery
import datetime

# Import the geemap (https://geemap.org/) module which
# has a visualisation tool
import geemap

# Geopandas allows us to read the shapefile used to
# define the region of interest (ROI)
import geopandas

# The colab module to access data from your google drive
from google.colab import drive

In [2]:
try:
  import pb_gee_tools
  import pb_gee_tools.datasets
  import pb_gee_tools.convert_types
except:
  !git clone https://github.com/remotesensinginfo/pb_gee_tools.git
  !pip install ./pb_gee_tools/.
  import pb_gee_tools
  import pb_gee_tools.datasets
  import pb_gee_tools.convert_types

Cloning into 'pb_gee_tools'...
remote: Enumerating objects: 358, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 358 (delta 42), reused 45 (delta 17), pack-reused 271 (from 1)
Receiving objects: 100% (358/358), 566.98 KiB | 4.76 MiB/s, done.
Resolving deltas: 100% (190/190), done.
Processing ./pb_gee_tools
  Preparing metadata (setup.py) ... done
  Created wheel for pb_gee_tools: filename=pb_gee_tools-0.3.0-py3-none-any.whl size=17966 sha256=5d398a4c3200234095d0553cf3f800b5ffbfd7e5ca579999745d5e9c25469bc7
  Stored in directory: /tmp/pip-ephem-wheel-cache-504zparw/wheels/e8/da/17/69dc01c6cbd07adb00923d8c1dd9a22fb31e96a17b74c93812
Successfully built pb_gee_tools


In [3]:
ee_prj_name = "ee-pb-dev"  # <==== Replace this with your own EE project string
ee.Authenticate()
ee.Initialize(project=ee_prj_name)

In [4]:
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
# The file path on google drive for ROI output vector file.
vec_tiles_file = "/content/drive/MyDrive/mangrove_chng_cls/cls_tiles.geojson"

# Start and End date
start_date = datetime.datetime(year=2020, month=1, day=1)
end_date = datetime.datetime(year=2020, month=12, day=31)

# Output no data value
no_data_val = 0.0

# Define the output directory on gdrive
out_gdrive_dir = "mangrove_chng_cls_tiles"

In [8]:
# Load the classifier from asset
cls_mdl_asset_id = f'projects/{ee_prj_name}/assets/mng_test_rf_cls'
rf_cls_mdl = ee.Classifier.load(cls_mdl_asset_id)

In [11]:
# Read the vector layer and make sure it is project using WGS84 (EPSG:4326)
vec_cls_roi_gdf = geopandas.read_file(vec_tiles_file).to_crs(4326)

vec_cls_roi_gdf.head()

from matplotlib import pyplot as plt
_df_3.plot(kind='scatter', x='index', y='fid', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_4.sort_values('index', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('tile_names')):
  _plot_series(series, series_name, i)
  fig.legend(title='tile_names', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['fid']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'fid'}, axis=1)
              .sort_values('fid', ascending=True))
  xs = counted['fid']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_5.sort_values('fid', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('tile_names')):
  _plot_series(series, series_name, i)
  fig.legend(title='tile_names', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('fid')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_6['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_7['fid'].plot(kind='line', figsize=(8, 4), title='fid')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_8['tile_names'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_8, x='index', y='tile_names', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_9['tile_names'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_9, x='fid', y='tile_names', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

,fid,tile_names,geometry
0,18544,n0915e0225,"POLYGON Z ((91.5 22.5 0, 92 22.5 0, 92 22 0, 9..."
1,18545,n0920e0225,"POLYGON Z ((92 22.5 0, 92.5 22.5 0, 92.5 22 0,..."
2,19264,n0915e0220,"POLYGON Z ((91.5 22 0, 92 22 0, 92 21.5 0, 91...."
3,19265,n0920e0220,"POLYGON Z ((92 22 0, 92.5 22 0, 92.5 21.5 0, 9..."
4,19984,n0915e0215,"POLYGON Z ((91.5 21.5 0, 92 21.5 0, 92 21 0, 9..."


In [12]:
tile_names = vec_cls_roi_gdf["tile_names"]
for tile_name in tile_names:
  print(tile_name)
  # Subset the input vector to just the tile geometry
  vec_tile_gdf = vec_cls_roi_gdf[vec_cls_roi_gdf["tile_names"] == tile_name]

  # Get layer bbox: minx, miny, maxx, maxy
  gp_roi_bbox = vec_tile_gdf.total_bounds

  # Create the GEE geometry from the bbox.
  roi_cls_west = gp_roi_bbox[0]
  roi_cls_east = gp_roi_bbox[2]
  roi_cls_north = gp_roi_bbox[3]
  roi_cls_south = gp_roi_bbox[1]
  cls_tile_aoi = ee.Geometry.BBox(roi_cls_west, roi_cls_south, roi_cls_east, roi_cls_north)

  # Load the Sentinel-2 imagery.
  s2_img_col = pb_gee_tools.datasets.get_sen2_sr_collection(
      aoi=cls_tile_aoi,
      start_date=start_date,
      end_date=end_date,
      cloud_thres = 70,
  )

  # Create a median composite clipped to the ROI.
  s2_med_image = s2_img_col.median().clip(cls_tile_aoi).unmask(no_data_val)

  # Get the list of image bands.
  img_bands = s2_med_image.bandNames()

  # Apply the Classification.
  cls_rslts_img = s2_med_image.classify(rf_cls_mdl)

  # Define the output file name
  out_cls_img_file = f"{tile_name}_asset_mdl_cls"
  """
  # Export the classification image to your Google Drive
  task = ee.batch.Export.image.toDrive(
          image=cls_rslts_img,
          description=out_cls_img_file,
          folder=out_gdrive_dir,
          scale=10,
          region=cls_tile_aoi,
          fileFormat="GeoTIFF",
          formatOptions={"cloudOptimized": True, 'noData': 0},
  )
  task.start()
  """

n0915e0225
n0920e0225
n0915e0220
n0920e0220
n0915e0215
n0920e0215
n0925e0215
n0930e0215
n0920e0210
n0925e0210
n0930e0210
n0920e0205
n0925e0205
n0930e0205
n0935e0205
n0925e0200
n0930e0200
n0935e0200
n0940e0200
n0930e0195
n0935e0195
n0940e0195
n0930e0190
n0935e0190
n0940e0190
n0940e0185
n0945e0185
n0940e0180
n0945e0180
n0940e0175
n0945e0175
n0940e0170
n0945e0170
n0940e0165
n0940e0160
